In [1]:
import time
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
persist_directory = "../dbv2"
embeddings = OpenAIEmbeddings()
faqdb = Chroma(
    persist_directory=persist_directory,
    embedding_function = embeddings
)
from langchain.prompts.prompt import PromptTemplate
template = """"Tickit is a rewards programme brought to you by Dubai Holding, offering an effortless instant way to earn and redeem points across your favourite brands and destinations in the UAE. The programme offers a comprehensive rewards experience across diverse categories including the emirate’s largest and most iconic attractions and entertainment destinations, leading theme parks, thriving dining and hospitality, online and in-store shopping, transportation and more. Tickit is the first rewards programme to use card-linked technology to earn and redeem points.
You are a customer service agent answering user queries about Tickit using the document below. 
If you can't find the answer in the document, say you don't know the answer and propose him to rephrase his query with more details.

Context: 
{context}\n\n

Current conversation:
{chat_history}

Question:
{question}\n\n

Answer:
"""
 
faq_prompt= PromptTemplate.from_template(template)
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
from langchain.chains import LLMChain,ConversationalRetrievalChain
faq_chain = ConversationalRetrievalChain.from_llm(llm=llm,retriever=faqdb.as_retriever(search_kwargs={"k": 5}),qa_prompt=faq_prompt)

Using embedded DuckDB with persistence: data will be stored in: ../dbv2


In [6]:
persist_directory = "../dboffersv2"
offerdb = Chroma(
    persist_directory=persist_directory,
    embedding_function = embeddings
)
template = """"Tickit is a rewards programme brought to you by Dubai Holding, offering an effortless instant way to earn and redeem points across your favourite brands and destinations in the UAE. The programme offers a comprehensive rewards experience across diverse categories including the emirate’s largest and most iconic attractions and entertainment destinations, leading theme parks, thriving dining and hospitality, online and in-store shopping, transportation and more.
You are a customer service agent answering user queries about brands listed in Tickit. The document below contains list of brands or outlets enrolled in Tickit where users can earn and redeem points. 
Reply as bullet points when possible.
If you can't find the answer in the document, say you are unable to find the answer and propose him to reach out to the human agent.

Document: 
{context}\n\n

Current conversation:
{chat_history}

Question:
{question}\n\n
"""
 
outlet_prompt= PromptTemplate.from_template(template)
offer_chain = ConversationalRetrievalChain.from_llm(llm=llm,retriever=offerdb.as_retriever(search_kwargs={"k": 5}),qa_prompt=outlet_prompt)

Using embedded DuckDB with persistence: data will be stored in: ../dboffersv2


In [7]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool

In [8]:
def faq_func(x):
    return faq_chain.run({"question":x,'chat_history':chat_history[-2:]})
def offer_func(x):
    return offer_chain.run({"question":x,'chat_history':chat_history[-2:]})
tools = [
    Tool(
        name = "FAQ Bot",
        func=faq_func,
        description="useful for when you need to Frequently Asked Questions about Tickit. Questions related to points, redemption, safety, data, payments, cards etc.  " ,
        return_direct=True
    ),
    Tool(
        name = "Brands and Offers Bot",
        func=offer_func,
        description="useful for when you need to answer questions about brands or outlets listed in Tickit loyalty in differnt locations",
        return_direct=True
    ),
]

In [9]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [10]:
chat_history = []
query = "Are there any dining options in Dubai mall?"
result = agent.run(input=query)
print(result)



> Entering new AgentExecutor chain...
 I need to find out what brands and offers are available in Dubai mall
Action: Brands and Offers Bot
Action Input: Dining options in Dubai mall
Observation: Answer:
At Dubai Mall, you can find the following dining options: 
- Eataly: Italian restaurant 
- Catch 22: Restaurant 
- Roadster Diner: American restaurant 
- Mitts and Trays: Restaurant


> Finished chain.
Answer:
At Dubai Mall, you can find the following dining options: 
- Eataly: Italian restaurant 
- Catch 22: Restaurant 
- Roadster Diner: American restaurant 
- Mitts and Trays: Restaurant


In [11]:
chat_history.append((query, result))
query = "Is linking cards safe? How is it protected?"
result = agent.run(input=query)
print(result)



> Entering new AgentExecutor chain...
 I need to find out more information about card linking safety
Action: FAQ Bot
Action Input: card linking safety
Observation: Tickit takes the security of your credit/debit card information very seriously. We use bank-grade security measures to protect your data, and all credit and debit card information is stored with our banking partner using their secure tokenization and vaulting systems. Additionally, you can activate biometrics to access the app for added security.


> Finished chain.
Tickit takes the security of your credit/debit card information very seriously. We use bank-grade security measures to protect your data, and all credit and debit card information is stored with our banking partner using their secure tokenization and vaulting systems. Additionally, you can activate biometrics to access the app for added security.


In [18]:
chat_history.append((query, result))
query = "Are there any pharmacies in tickit?"
result = agent.run(input=query)
print(result)



> Entering new AgentExecutor chain...
 I need to find out what brands and outlets are listed in Tickit
Action: Brands and Offers Bot
Action Input: Pharmacies
Observation: Answer: Tickit loyalty pharmacies are located across the UAE, including Dubai Mall, Dubai Hills Mall, MOE, The Beach, and City Walk.


> Finished chain.
Answer: Tickit loyalty pharmacies are located across the UAE, including Dubai Mall, Dubai Hills Mall, MOE, The Beach, and City Walk.


In [19]:
chat_history.append((query, result))
query = "Any of those in Dubai Mall?"
result = agent.run(input=query) 
print(result)



> Entering new AgentExecutor chain...
 I need to find out which brands and outlets are listed in Tickit loyalty in Dubai Mall.
Action: Brands and Offers Bot
Action Input: Brands and outlets in Dubai Mall
Observation: Answer: Brands and outlets located in Dubai Mall include: 
- Centrepoint 
- Palladium 
- Sunglass Hut 
- DKNY 
- Samsonite 
- Babyshop 
- Splash 
- Lifestyle 
- Shoe Mart 
- Ray-Ban 
- Oakley 
- Beverly Hills Polo Club 
- ASICS 
- La Brioche 
- Akhom Consulting 
- Palladium Hotel Group 
- Jamie Italian


> Finished chain.
Answer: Brands and outlets located in Dubai Mall include: 
- Centrepoint 
- Palladium 
- Sunglass Hut 
- DKNY 
- Samsonite 
- Babyshop 
- Splash 
- Lifestyle 
- Shoe Mart 
- Ray-Ban 
- Oakley 
- Beverly Hills Polo Club 
- ASICS 
- La Brioche 
- Akhom Consulting 
- Palladium Hotel Group 
- Jamie Italian
